In [4]:
from string import ascii_letters
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import scipy as sp
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import ensemble
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.layers.core import Reshape
from keras.wrappers.scikit_learn import KerasClassifier
import re
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from time import clock
from collections import defaultdict
def toVector(x_train, x_test):
    print x_train.shape
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 2))
    # Vectorize the samples.
    frames = [x_train, x_test]
    result = pd.concat(frames)
    ds = vectorizer.fit_transform(result)
    #x_test = vectorizer.fit_transform(result)
    #print x_train.shape, x_test.shape
    xx_train = ds[:x_train.shape[0]][:]
    xx_test = ds[x_train.shape[0]:][:]
    print xx_train.shape, xx_test.shape, type(xx_train)
    xx_train = pd.DataFrame(xx_train.todense())
    xx_test = pd.DataFrame(xx_test.todense())
    print xx_train.shape, xx_test.shape
    return xx_train, xx_test
    #return pd.DataFrame(eval(x_train)), pd.DataFrame(eval(x_test))
def importData():
    xy_train = pd.read_csv('evo_train.csv', delimiter=',', header=None)
    print xy_train.shape
    x_train = xy_train[:][0]
    y_train = xy_train[:][1]
    x_test = pd.read_csv('evo_test.csv', delimiter=',', header=None)[:][0]
    print x_train.shape, y_train.shape, x_test.shape
    y_tr1=open('y_tr1.csv', 'w')
    for y in y_train:
        y_tr1.write(str(y)+'\n')
    y_tr1.close()
    return x_train, y_train, x_test
def importData2():
    x_train = pd.read_csv('x_tr.csv', delimiter=',', header=None)[:][0]
    x_test = pd.read_csv('x_te.csv', delimiter=',', header=None)[:][0]
    return x_train, x_test
def convertToMin(x_train, x_test, n):
    words = {}
    for strW in x_train:
        word1=strW.split(' ')
        for word in word1:
            if (not words.has_key(word)):
                words[word]=1
            else:
                words[word]+=1
    for strW in x_test:
        word1=strW.split(' ')
        for word in word1:
            if (not words.has_key(word)):
                words[word]=1
            else:
                words[word]+=1
    x_tr=[]
    i=1
    for strW in x_train:
        word1=strW.split(' ')
        maxI=0
        mmaxI=0
        maxW="1"
        mmaxW="1"
        for word in word1:
            if (len(word)>3):
                #print len(word), (word), (words[word])
                if (words[word]>maxI):
                    mmaxW=maxW
                    mmaxI=maxI
                    maxW=word
                    maxI=words[word]
        x_tr.append(maxW+" "+mmaxW)
        #x_tr.append(maxW)
        i+=1
        #print maxW
        #if (i>100):
        #    break
    x_te=[]
    for strW in x_test:
        word1=strW.split(' ')
        maxI=0
        mmaxI=0
        maxW="1"
        mmaxW="1"
        for word in word1:
            if (len(word)>3):
                #print len(word), (word), (words[word])
                if (words[word]>maxI):
                    mmaxW=maxW
                    mmaxI=maxI
                    maxW=word
                    maxI=words[word]
        x_te.append(maxW+" "+mmaxW)
        #x_te.append(maxW)
        i+=1
        #print maxW
        #if (i>100):
        #    break
    x_tr1=open('x_tr2.csv', 'w')
    for x in x_tr:
        x_tr1.write(str(x)+'\n')
    x_tr1.close()
    x_te1=open('x_te2.csv', 'w')
    for x in x_te:
        x_te1.write(str(x)+'\n')
    x_te1.close()
def importData3():
    x_train = pd.read_csv('x_tr2.csv', delimiter=',', header=None)[:][0]
    y_train = pd.read_csv('y_tr1.csv', delimiter=',', header=None)[:][0]
    x_test = pd.read_csv('x_te2.csv', delimiter=',', header=None)[:][0]
    return x_train, y_train, x_test
def importDatans():
    x_train = pd.read_csv('x_trns.csv', delimiter=',', header=None)[:][0]
    y_train = pd.read_csv('y_tr1.csv', delimiter=',', header=None)[:][0]
    x_test = pd.read_csv('x_tens.csv', delimiter=',', header=None)[:][0]
    return x_train, y_train, x_test
def convertDataset(x_tr, x_te):
    for i in range (x_tr.shape[0]):
        x_tr[i]=x_tr[i].decode('utf-8').lower()
        newstr=[]
        for j in range (len(x_tr[i])):
            if (not (ord(x_tr[i][j]) in range(ord('0'), ord('9')+1) or ord(x_tr[i][j]) in range(ord(u'а'), ord(u'я')+1) or ord(x_tr[i][j]) in range(ord('a'), ord('z')+1) or x_tr[i][j]==' ')):
                newstr.append(" ")
                continue
            newstr.append(x_tr[i][j].decode('utf-8'))
        newstr = ''.join(newstr)
        newstr=re.sub(r'\s+', '', newstr)
        x_tr[i]=newstr
    for i in range (x_te.shape[0]):
        x_te[i]=x_te[i].decode('utf-8').lower()
        newstr=[]
        for j in range (len(x_te[i])):
            if (not (ord(x_te[i][j]) in range(ord('0'), ord('9')+1) or ord(x_te[i][j]) in range(ord(u'а'), ord(u'я')+1) or ord(x_te[i][j]) in range(ord('a'), ord('z')+1) or x_te[i][j]==' ')):
                newstr.append(" ")
                continue
            newstr.append(x_te[i][j].decode('utf-8'))
        newstr = ''.join(newstr)
        newstr=re.sub(r'\s+', '', newstr)
        x_te[i]=newstr
    x_tr1=open('x_trns.csv', 'w')
    for x in x_tr:
        x_tr1.write(str(x)+'\n')
    x_tr1.close()
    x_te1=open('x_tens.csv', 'w')
    for x in x_te:
        x_te1.write(str(x)+'\n')
    x_te1.close()
    return x_tr, x_te
def LSTMRes(X_train, X_test, y_train):
    y_train = y_train.as_matrix()
    X_train = X_train.as_matrix()
    X_test = X_test.as_matrix()
    print X_train.shape, X_test.shape, y_train.shape
    print X_train.shape, y_train.shape
    model = Sequential()
    model.add(Dense(1000, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(61, activation='softmax'))

    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=20, batch_size=128)  
    zLSMT = model.predict_classes(X_test)
    y_res=open('y_testLSTM.csv', 'w')
    for i in range (len(zLSMT)):
        y_res.write(str(i)+","+str(zLSMT[i])+"\n")
    y_res.close()
    print "NN done"
    return zLSMT
def xgboostRes(X_train, X_test, y_train):
    param = {'max_depth':4, 'silent':1, 'objective':'binary:logistic'}
    bst = XGBClassifier(**param)
    print X_train.shape, y_train.shape
    bst.fit(X_train, y_train)
    print "fit done"
    zG = bst.predict(X_test)
    y_res=open('y_testXGB.csv', 'w')
    #y_res.write('\n'.join(str(v) for v in zG))   
    y_res.write("id,GROUP_ID\n")
    for i in range (len(zG)):
    #for i in range (37240):
        #y_res.write(str(i)+",14\n")
        y_res.write(str(i)+","+str(zG[i])+"\n")
    y_res.close()
    print "xgb done"
def main():
    nnnn=5000
    #x_train, y_train, x_test = importData()
    #x_train, x_test = convertDataset(x_train, x_test)
    #x_train, x_test = importData2()
    #convertToMin(x_train, x_test, 1)
    #x_train, y_train, x_test = importData3()
    x_train, y_train, x_test = importDatans()
    print x_train.shape, y_train.shape, x_test.shape
    x_train, x_test = toVector(x_train[:][:], x_test[:][:])
    #x_train, x_test = toVector(x_train[:nnnn][:], x_test[:nnnn][:])
    #LSTMRes(x_train, x_test, y_train[:nnnn])
    LSTMRes(x_train, x_test, y_train[:])
    #print "xg"
    #st = clock()
    #xgboostRes(x_train[:nnnn][:], x_test[:nnnn][:], y_train[:nnnn])
    #print clock()-st
    #xgboostRes(x_train[:][:], x_test[:][:], y_train[:])
main()

(55860,) (55860,) (37240,)
(55860,)
(55860, 4208) (37240, 4208) <class 'scipy.sparse.csr.csr_matrix'>
(55860, 4208) (37240, 4208)
(55860, 4208) (37240, 4208) (55860,)
(55860, 4208) (55860,)
Epoch 1/20
55860/55860 [==============================] - 86s - loss: 1.3042 - acc: 0.6545    
Epoch 2/20
55860/55860 [==============================] - 74s - loss: 0.6538 - acc: 0.8206    
Epoch 3/20
55860/55860 [==============================] - 74s - loss: 0.4513 - acc: 0.8763    
Epoch 4/20
55860/55860 [==============================] - 73s - loss: 0.3185 - acc: 0.9122    
Epoch 5/20
55860/55860 [==============================] - 75s - loss: 0.2222 - acc: 0.9375    
Epoch 6/20
55860/55860 [==============================] - 77s - loss: 0.1552 - acc: 0.9558    
Epoch 7/20
55860/55860 [==============================] - 77s - loss: 0.1141 - acc: 0.9670    
Epoch 8/20
55860/55860 [==============================] - 78s - loss: 0.0876 - acc: 0.9745    
Epoch 9/20
55860/55860 [==========================